<a href="https://colab.research.google.com/github/Hugekyung/TIL/blob/master/Crawl/%EB%89%B4%EC%8A%A4%ED%81%AC%EB%A1%A4%EB%A7%81_scrapycode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import scrapy
import re
from datetime import datetime
import pandas as pd
import time
import json
import requests
# 네이버 뉴스 '사회' 카테고리
# 기사 수 TOP3 연합뉴스, 뉴시스, 뉴스1
# 연합뉴스 https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=001&date={}
# 뉴시스 https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=003&date={}
# 뉴스1 https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=421&date={}

class CrawlNews(scrapy.Spider):
    name = "news"
    allowed_domains = ["news.naver.com", "tts.news.naver.com"]
    url_format = "https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=421&date={}"


    def __init__(self):
        start = input("시작 날짜를 입력하세요(ex.2020-09-30): ")
        end = input("마지막 날짜를 입력하세요(ex.2020-09-30): ")
        startdate =  datetime.strptime(start, "%Y-%m-%d")
        enddate =  datetime.strptime(end, "%Y-%m-%d")

        self.start_urls= [] # startdate~enddate 사이 날짜에 해당하는 start_url들을 모두 저장
        for cur_date in pd.date_range(startdate, enddate):
            self.start_urls.append(self.url_format.format(cur_date.strftime("%Y%m%d")))


    def start_requests(self):
        for start_url in self.start_urls: # 가져올 페이지를 리스트에 저장
            yield scrapy.Request(url=start_url, callback=self.parse, meta={"start_url" : start_url, "page_num": 1})
            time.sleep(5)


    def parse(self, response):
        page_num = response.meta["page_num"]
        if int(response.css("div.paging strong::text").get()) != page_num: # 뉴스 페이지 번호와 for문을 도는 page_num이 일치하지 않으면 중지
            return # 함수 종료
        else:
            for item in response.css("ul.type06_headline li"):
                url = item.css("a::attr(href)").get()
                yield scrapy.Request(url, callback=self.parse_detail)

            for item2 in response.css("ul.type06 li"):
                url2 = item2.css("a::attr(href)").get()
                yield scrapy.Request(url2, callback=self.parse_detail)
                
        # 다음 페이지 url
        page_num += 1
        start_url = response.meta["start_url"]
        next_page = start_url + "&page={}".format(page_num)
        print("다음 페이지============", next_page)
        yield scrapy.Request(next_page, callback=self.parse, meta={"start_url" : start_url, "page_num" : page_num})
    

    def parse_detail(self, response):
        try:
            if response.css("div.media_end_head_autosummary._auto_summary_wrapper > a::text").get() == '요약봇': # 요약봇 버튼이 있는 경우만 크롤링
                upload_date = response.css("div.sponsor span.t11::text").get()
                media = response.css("div.press_logo img::attr(title)").get()
                title = response.css("h3#articleTitle::text").get()
                text = ''.join(response.css("div#articleBodyContents::text").getall()).replace("\n","").strip()
            yield {
                    "upload_date": upload_date,
                    "media": media,
                    "title": title,
                    "text": text
            }
        except:
            pass